In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import vdmlab as vdm

In [17]:
tuning_curve = np.array([[3., 0., 0.]])
counts = np.array([[0., 2., 4.]])
binsize = 1.0
prob = vdm.bayesian_prob(counts, tuning_curve, binsize)
prob

array([[ nan,  nan,  nan],
       [  1.,  nan,  nan],
       [  1.,  nan,  nan]])

In [21]:
tuning_curve = np.array([[10.]])
counts = np.array([[0., 2., 4.]])
binsize = 1.0
prob = vdm.bayesian_prob(counts, tuning_curve, binsize)
prob[0][0]
np.isnan(prob[0][0])

True

In [22]:
assert np.isnan(prob[0][0])

In [59]:
tuning_curve = np.array([[2., 0., 0.],
                             [0., 5., 0.],
                             [0., 0., 5.]])
counts = np.array([[0., 8, 0.],
                   [4., 0., 1.],
                   [0., 1., 3.]])
binsize = 1.0
prob = vdm.bayesian_prob(counts, tuning_curve, binsize)
prob

array([[  3.10880460e-02,   9.67364171e-01,   1.54778267e-03],
       [  9.98834476e-01,   1.94254064e-04,   9.71270319e-04],
       [  1.33827265e-01,   3.33143360e-02,   8.32858399e-01]])

In [62]:
np.allclose(prob[0], np.array([0.0310880460, 0.967364171, 0.00154778267]))
np.allclose(prob[1], np.array([0.998834476, 0.000194254064, 0.000971270319]))
np.allclose(prob[2], np.array([0.133827265, 0.0333143360, 0.832858399]))

True

In [ ]:
assert np.allclose(prob, np.array([[0.000168461312, 0.499915769, 0.499915769]]))

In [ ]:
a = [False, True, False]

In [ ]:
if any(a):
    print('yes')

In [ ]:
decoded_position = vdm.decode_location(prob, linear)

In [ ]:
decoded_position

In [ ]:
decoded = dict()
decoded['time'] = centers
decoded['position'] = decoded_position

In [ ]:
linear = dict()
# linear['position'] = np.array([0., 5., 10., 10., 5., 0.])[:2]
linear['time'] = np.array([0., 1., 2., 3., 4., 5.])[:2]

In [ ]:
sequences = vdm.decoded_sequences(decoded, min_idx_length=1, max_idx_jump=10)

In [ ]:
sequences

In [ ]:
tuning_curves = np.array([[1., 0.1, 1.],
                         [0.1, 1., 0.1],
                         [1., 1., 1.],
                          [0.1, 0.1, 0.1],
                          [0.1, 1., 0.1]])

In [ ]:
counts = np.array([[0., 0., 0., 0., 0., 0.],
                   [0., 0., 1., 0., 0., 0.],
                   [2., 0., 1., 1., 0., 1.],
                   [0., 1., 0., 0., 2., 0.],
                   [1., 2., 1., 2., 1., 2.]])

In [ ]:
count_matrix = np.array([[3., 2., 1.],
                         [0., 0., 0.],
                         [1., 0., 2.],
                         [0., 2., 0.]])

In [ ]:
activity_matrix = bool_counts(count_matrix)
prob_active = prob_active_neuron(activity_matrix)
prob_expected = expected_cooccur(prob_active)
prob_observed = observed_cooccur(activity_matrix)
prob_shuffle = shuffle_cooccur(activity_matrix, num_shuffles=100)
# prob_zscore = zscore_cooccur(prob_observed, prob_shuffle)

In [ ]:
%time
prob_zscore = zscore_cooccur(prob_observed, prob_shuffle)

In [ ]:
prob_zscore

In [ ]:
linear = dict()
linear['position'] = np.concatenate((np.linspace(0, 10, 10), np.linspace(10, 0, 10), np.linspace(0, 10, 20)))
linear['time'] = np.linspace(0, 100, 40)

In [ ]:
plt.plot(linear['time'], linear['position'])
plt.show()

In [ ]:
import random
random.seed(0)
spikes = dict(time=[[], [], [], []])
for i in range(10):
    spikes['time'][0].append(random.uniform(0, 100))
for i in range(15):
    spikes['time'][1].append(random.uniform(20, 30))
for i in range(15):
    spikes['time'][2].append(random.uniform(60, 80))   
for i in range(15):
    spikes['time'][3].append(random.uniform(5, 10))
for i in range(5):
    for neuron_spikes in spikes['time']:
        neuron_spikes.append(random.uniform(0, 100))

In [ ]:
for idx, neuron_spikes in enumerate(spikes['time']):
    plt.plot(neuron_spikes, np.ones(len(neuron_spikes))+idx, '|', color='k', ms=10, mew=1)
plt.ylim(0.5, 4.5)
plt.show()

In [ ]:
tuning = vdm.tuning_curve(linear, spikes, sampling_rate=1, binsize=3, filter_type=None)

In [ ]:
colours = ['b', 'g', 'r', 'c']

x = list(range(0, np.shape(tuning)[1]))
for i, neuron_tc in enumerate(tuning):
    plt.plot(neuron_tc, color=colours[i])
#     plt.fill_between(x, 0, neuron_tc, facecolor=colours[i])
plt.show()

In [ ]:
tuning